In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
import wrds
import talib as ta
import datetime, time

from scipy import stats
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [4]:
# load return data
crsp = pd.read_csv('data/return.zip', 
                   parse_dates=['date'], infer_datetime_format=True)

E:\ProgramData\Anaconda3\envs\quant\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
crsp[['PERMNO', 'PERMCO']] = crsp[['PERMNO', 'PERMCO']].astype(int)

In [6]:
crsp_name = pd.read_csv('data/crsp_name.csv', 
                   parse_dates=['namedt','nameendt'], infer_datetime_format=True)
crsp_name = crsp_name.drop('Unnamed: 0', axis=1)

In [79]:
crsp_name[['permno', 'shrcd', 'exchcd']] = \
crsp_name[['permno', 'shrcd', 'exchcd']].astype(int)

crsp_name['namedt'] = pd.to_datetime(crsp_name['namedt'])
crsp_name['nameendt'] = pd.to_datetime(crsp_name['nameendt'])
# if nameendt is missing then set to today date
crsp_name['nameendt'] = crsp_name['nameendt'].fillna(pd.to_datetime('today'))

In [80]:
crsp_merge = pd.merge(crsp, crsp_name, 'inner', left_on='PERMNO', right_on='permno')
crsp_merge = crsp_merge[(crsp_merge.date <= crsp_merge.nameendt) &
                       (crsp_merge.date >= crsp_merge.namedt)]

In [82]:
crsp_merge = crsp_merge.drop(['permno', 'namedt', 'nameendt'], axis=1)

In [91]:
crsp_merge.RET = crsp_merge.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
crsp_merge.RETX = crsp_merge.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)

crsp_merge.RET = crsp_merge.RET.fillna(0)
crsp_merge.RETX = crsp_merge.RETX.fillna(0)

In [59]:
# db = wrds.Connection(wrds_username='kpu2019')
# ccm = db.get_table('crsp', 'ccmxpf_lnkhist', date_cols=['linkdt', 'linkenddt'])
# db.close()
# ccm.to_csv('data/ccm_hist.csv')

Loading library list...
Done


In [74]:
ccm = pd.read_csv('data/ccm_hist.csv')

In [98]:
ccm = ccm.drop('Unnamed: 0', axis=1)

In [99]:
ccm.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,linkdt,linkenddt
0,1000,C,00X,NU,NaN,NaN,1961-01-01,1970-09-29
1,1000,P,01,NU,NaN,NaN,1970-09-30,1970-11-12
2,1000,P,01,LU,25881.0,23369.0,1970-11-13,1978-06-30
3,1001,C,00X,NU,NaN,NaN,1978-01-01,1983-09-19
4,1001,P,01,LU,10015.0,6398.0,1983-09-20,1986-07-31


In [100]:
ccm = ccm[((ccm.linktype == 'LU') | (ccm.linktype == 'LC')) & \
   ((ccm.linkprim == 'P') | (ccm.linkprim == 'C'))]

In [101]:
ccm['linkdt']=pd.to_datetime(ccm['linkdt'])
ccm['linkenddt']=pd.to_datetime(ccm['linkenddt'])
# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm.columns = ['gvkey', 'linkprim', 'liid', 'linktype', 'permno', 'permco', 'linkdt',
       'linkenddt']

In [210]:
comp = pd.read_csv('data/fundamental.zip', 
                   parse_dates=['datadate'], infer_datetime_format=True)
comp = comp.drop(['indfmt', 'consol', 'popsrc', 'datafmt'], axis=1)

resample by fiscal year

In [117]:
data_aol = comp.query('tic == "AOL"')
data_aol = data_aol.reset_index().drop('index', axis=1)

In [124]:
data_aol.head()

,gvkey,datadate,fyearq,fqtr,tic,cusip,conm,curcdq,datacqtr,datafqtr,dlttq,oiadpq,pstkq,exchg,costat,mkvaltq
0,183920,2007-03-31,2007,1,AOL,00184X105,AOL INC,USD,2007Q1,2007Q1,NaN,NaN,NaN,11.0,I,NaN
1,183920,2007-06-30,2007,2,AOL,00184X105,AOL INC,USD,2007Q2,2007Q2,NaN,NaN,NaN,11.0,I,NaN
2,183920,2007-09-30,2007,3,AOL,00184X105,AOL INC,USD,2007Q3,2007Q3,NaN,NaN,NaN,11.0,I,NaN
3,183920,2007-12-31,2007,4,AOL,00184X105,AOL INC,USD,2007Q4,2007Q4,24.7,1468.0,0.0,11.0,I,NaN
4,183920,2008-03-31,2008,1,AOL,00184X105,AOL INC,USD,2008Q1,2008Q1,NaN,NaN,NaN,11.0,I,NaN


In [155]:
(data_aol[data_aol.fqtr == 1].datadate + MonthBegin(-3)).\
apply(lambda x : x.strftime('%b')).unique()

array(['Jan'], dtype=object)

In [173]:
data_aol[data_aol.fqtr == 4].datadate.apply(lambda x : x.strftime('%b')).unique()

array(['Dec'], dtype=object)

In [163]:
data_aol.set_index('datadate').resample('A-Dec').mean()

,gvkey,fyearq,fqtr,dlttq,oiadpq,pstkq,exchg,mkvaltq
datadate,,,,,,,,
2007-12-31,183920.0,2007.0,2.5,24.7,1468.0,0.0,11.0,NaN
2008-12-31,183920.0,2008.0,2.5,33.7,358.8,0.0,11.0,NaN


In [168]:
data_aol.datadate.apply(lambda x : x.strftime('%b'))

0    Mar
1    Jun
2    Sep
3    Dec
4    Mar
5    Jun
6    Sep
7    Dec
Name: datadate, dtype: object

In [148]:
data_aol.set_index('datadate').resample('Q-Jan').mean()

,gvkey,fyearq,fqtr,dlttq,oiadpq,pstkq,exchg,mkvaltq
datadate,,,,,,,,
2007-04-30,183920,2007,1,NaN,NaN,NaN,11.0,NaN
2007-07-31,183920,2007,2,NaN,NaN,NaN,11.0,NaN
2007-10-31,183920,2007,3,NaN,NaN,NaN,11.0,NaN
2008-01-31,183920,2007,4,24.7,1468.0,0.0,11.0,NaN
2008-04-30,183920,2008,1,NaN,NaN,NaN,11.0,NaN
2008-07-31,183920,2008,2,NaN,526.4,NaN,11.0,NaN
2008-10-31,183920,2008,3,NaN,265.3,NaN,11.0,NaN
2009-01-31,183920,2008,4,33.7,284.7,0.0,11.0,NaN


首先通过comp中的datadate数据提取出fiscal year-end month

先验证下year-end month对于每个gvkey来说,是否为唯一, 统计个数即可

year-end month个数只有1的公司筛选出来, 然后对原来的comp进行筛选

In [212]:
comp = comp.sort_values(by=['gvkey','datadate'])
comp1 = comp.copy()
# comp1 = comp1.set_index('datadate')

In [223]:
comp.shape

(1217797, 16)

In [222]:
comp1.shape

(1217797, 14)

In [335]:
def retrieve_year_start_count(df):
    res = len(df[df.fqtr == 1].datadate.\
        apply(lambda x : x.strftime('%b')).unique())
    return res

def retrieve_year_start_month(df):
#     res = len(df[df.fqtr == 4].datadate.apply(lambda x : x.strftime('%b')).unique())
#     if output == 'count':
#         res = len((df[df.fqtr == 1].datadate + MonthBegin(-3)).\
#             apply(lambda x : x.strftime('%b')).unique())
#         return res
#     elif output == 'string':
    res = (df[df.fqtr == 1].datadate + MonthBegin(-3)).\
        apply(lambda x : 'AS-' + x.strftime('%b')).unique()[0]
    return res

def retrieve_year_end_month(df):
    res = (df[df.fqtr == 1].datadate + MonthEnd(9)).\
        apply(lambda x : 'A-' + x.strftime('%b')).unique()[0]
    return res

In [324]:
datetime.datetime.strptime('2018-03-31', '%Y-%m-%d') + MonthBegin(-3)

Timestamp('2018-01-01 00:00:00')

In [333]:
(datetime.datetime.strptime('2018-07-31', '%Y-%m-%d') + MonthEnd(9)).strftime('%b')

'Apr'

In [321]:
data_aol.set_index('datadate').resample('AS-Feb').mean()

,gvkey,fyearq,fqtr,dlttq,oiadpq,pstkq,exchg,mkvaltq
datadate,,,,,,,,
2007-02-01,183920.0,2007.0,2.5,24.7,1468.0,0.0,11.0,NaN
2008-02-01,183920.0,2008.0,2.5,33.7,358.8,0.0,11.0,NaN


In [198]:
g = comp1.groupby('gvkey')
fiscal_month_count = g.apply(retrieve_year_start_count)
code_list = fiscal_month_count[fiscal_month_count == 1].index.tolist()
comp1 = comp1.reset_index().set_index(['gvkey'])
comp2 = comp1.loc[code_list]
comp2 = comp2.reset_index()

In [350]:
fiscal_month_count.to_csv('fiscal_end_month_list.csv')

In [246]:
comp2.head()

,gvkey,datadate,fyearq,fqtr,tic,cusip,conm,curcdq,datacqtr,datafqtr,dlttq,oiadpq,pstkq,exchg,costat,mkvaltq
0,1000,1973-03-31,1973,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q1,1973Q1,7.0,NaN,0.0,12.0,I,NaN
1,1000,1973-06-30,1973,2,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q2,1973Q2,7.0,NaN,0.0,12.0,I,NaN
2,1000,1973-09-30,1973,3,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q3,1973Q3,7.0,NaN,0.0,12.0,I,NaN
3,1000,1973-12-31,1973,4,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q4,1973Q4,7.0,3.837,0.0,12.0,I,NaN
4,1000,1974-03-31,1974,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1974Q1,1974Q1,7.0,NaN,0.0,12.0,I,NaN


In [ ]:
comp2.query('gvkey == 5568')
comp.query('gvkey == 9559')#.to_csv('data/gvkey_9559.csv')

In [336]:
fiscal_end_month = comp2.groupby('gvkey').apply(retrieve_year_end_month)
fiscal_end_month.name = 'fs_month'
# fiscal_end_month = 'A-' + fiscal_end_month
fiscal_end_month = pd.DataFrame(fiscal_end_month)
# fiscal_end_month = pd.DataFrame(fiscal_end_month.fs_month.apply(lambda x  : x[2:]))

In [352]:
fiscal_end_month.to_csv('fiscal_end_month_list_unique.csv')

In [344]:
comp3 = pd.merge(comp2, fiscal_end_month, on=['gvkey'])
comp3 = comp3.set_index('datadate')
# comp4 = comp3.groupby()

In [345]:
def resample_fiscal_year(df):
    fsm = df.fs_month.iloc[0]
    res = df.resample(fsm).mean()
    return res

In [353]:
comp3.shape

(1046937, 16)

In [354]:
ccm.shape

(29210, 8)

In [346]:
comp3.loc[comp3.index.tolist()[:10]].groupby('gvkey').apply(resample_fiscal_year)

gvkey  fyearq  fqtr       dlttq     oiadpq      pstkq  \
gvkey  datadate                                                               
1000   1973-12-31    1000.0  1973.0   2.5    7.000000    3.83700    0.00000   
       1974-12-31    1000.0  1974.0   2.5    7.000000    3.49400    0.10350   
       1975-12-31    1000.0  1975.0   1.5    7.000000        NaN    0.41400   
1010   1973-12-31    1010.0  1973.0   2.5  194.473500   54.62800    0.00000   
       1974-12-31    1010.0  1974.0   2.5  216.934250   68.89700    0.00000   
       1975-12-31    1010.0  1975.0   1.5  245.920000   15.86150    0.00000   
1020   1973-12-31    1020.0  1973.0   2.5   45.504000    1.25450    0.00000   
       1974-12-31    1020.0  1974.0   2.5   43.026000   -0.97975    0.00000   
       1975-12-31    1020.0  1975.0   1.5   42.146000   -1.74400    0.00000   
1031   1973-03-31    1031.0  1972.0   4.0    4.474000    4.58000    0.00000   
       1974-03-31    1031.0  1973.0   2.5    7.995750    1.69700    0.00000   
       1975-03-31    1031.0  1974.0   2.5    7.410750    8.53400    0.00000   
       1976-03-31    1031.0  1975.0   1.0    4.078000        NaN    0.00000   
1039   1973-12-31    1039.0  1973.0   2.5    9.254000    4.41400    0.00000   
       1974-12-31    1039.0  1974.0   2.5   10.140500    3.46600    0.00000   
       1975-12-31    1039.0  1975.0   1.5   10.000000        NaN    0.00000   
1040   1973-12-31    1040.0  1973.0   2.5  152.247750  102.01000    1.66025   
       1974-12-31    1040.0  1974.0   2.5  169.561250   14.69225    1.64850   
       1975-12-31    1040.0  1975.0   1.5  198.360000   22.82850    1.64850   
1045   1973-12-31    1045.0  1973.0   2.5  618.095750   -9.19425    0.00000   
       1974-12-31    1045.0  1974.0   2.5  503.976500   11.30750    0.00000   
       1975-12-31    1045.0  1975.0   1.5  465.452000  -14.98650    0.00000   
1067   1973-09-30    1067.0  1973.0   3.0    5.384333   -0.00700    0.00000   
       1974-09-30    1067.0  1974.0   2.5    4.828250    1.07500    0.00000   
       1975-09-30    1067.0  1975.0   2.0    4.098333    0.50650    0.00000   
1070   1973-12-31    1070.0  1973.0   2.5    7.911000    0.66800    0.00000   
       1974-12-31    1070.0  1974.0   2.5    8.056250    3.49900    0.00000   
       1975-12-31    1070.0  1975.0   1.5   13.256000    3.08750    0.00000   
1074   1973-12-31    1074.0  1973.0   2.5   28.715750   12.25300    5.40575   
       1974-12-31    1074.0  1974.0   2.5   32.985750   -4.04800    5.40600   
...                     ...     ...   ...         ...        ...        ...   
65090  1975-12-31   65090.0  1975.0   1.5  418.961500        NaN  133.50000   
65091  1973-12-31   65091.0  1973.0   2.5  480.000000        NaN  114.00000   
       1974-12-31   65091.0  1974.0   2.5  536.288000        NaN  114.00000   
       1975-12-31   65091.0  1975.0   1.5  513.798500        NaN  114.00000   
65094  1973-12-31   65094.0  1973.0   2.5   85.640000        NaN   18.59000   
       1974-12-31   65094.0  1974.0   2.5   85.926000        NaN   18.59000   
       1975-12-31   65094.0  1975.0   1.5   85.953500        NaN   18.59000   
65095  1973-12-31   65095.0  1973.0   2.5  186.819000        NaN   51.20000   
       1974-12-31   65095.0  1974.0   2.5  196.932250        NaN   54.95000   
       1975-12-31   65095.0  1975.0   1.5  196.149000        NaN   66.20000   
65298  1973-12-31   65298.0  1973.0   2.5  128.870000        NaN   34.87600   
       1974-12-31   65298.0  1974.0   2.5  130.136750        NaN   34.87600   
       1975-12-31   65298.0  1975.0   1.5  124.906500        NaN   39.87600   
65351  1973-12-31   65351.0  1973.0   2.5   11.479000   15.12250    2.55275   
       1974-12-31   65351.0  1974.0   2.5   11.000250   17.15600    1.84200   
       1975-12-31   65351.0  1975.0   1.5    9.806000   17.82450    1.29600   
66430  1973-12-31   66430.0  1973.0   2.5  835.532500   81.82750    0.00000   
       1974-12-31   66430.0  1974.0   2.5  893.877500   81.46250    0.00

In [356]:
comp3.head(1)

,gvkey,fyearq,fqtr,tic,cusip,conm,curcdq,datacqtr,datafqtr,dlttq,oiadpq,pstkq,exchg,costat,mkvaltq,fs_month
datadate,,,,,,,,,,,,,,,,
1973-03-31,1000,1973,1,AE.2,000032102,A & E PLASTIK PAK INC,USD,1973Q1,1973Q1,7.0,NaN,0.0,12.0,I,NaN,A-Dec


In [358]:
comp4 = comp3.reset_index()

In [359]:
comp4.columns

Index(['datadate', 'gvkey', 'fyearq', 'fqtr', 'tic', 'cusip', 'conm', 'curcdq',
       'datacqtr', 'datafqtr', 'dlttq', 'oiadpq', 'pstkq', 'exchg', 'costat',
       'mkvaltq', 'fs_month'],
      dtype='object')

In [367]:
ccm1=pd.merge(comp4[['gvkey','datadate','fs_month']],ccm,how='left',on=['gvkey'])
ccm1['yearend']=ccm1['datadate']+YearEnd(0)
ccm1['jdate']=ccm1['yearend']+MonthEnd(6)

# set link date bounds
ccm2=ccm1[(ccm1['jdate']>=ccm1['linkdt'])&(ccm1['jdate']<=ccm1['linkenddt'])]
# ccm2=ccm2[['gvkey','permno','datadate','yearend', 'jdate', 'fs_month']]
ccm2=ccm2[['gvkey','permno','fs_month']]

In [368]:
ccm2.shape

(715915, 3)

In [369]:
ccm2.head()

,gvkey,permno,fs_month
0,1000,25881.0,A-Dec
1,1000,25881.0,A-Dec
2,1000,25881.0,A-Dec
3,1000,25881.0,A-Dec
4,1000,25881.0,A-Dec


In [371]:
crsp_merge.head()

,PERMNO,date,TICKER,COMNAM,PERMCO,CUSIP,PRC,RET,SHROUT,RETX,shrcd,exchcd
0,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,30,3
5,14403,1973-02-28,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-30.625,0.157272,21952.0,0.129032,30,3
10,14403,1973-03-30,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-35.125,0.146939,21952.0,0.146939,30,3
15,14403,1973-04-30,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-36.500,0.039146,21952.0,0.039146,30,3
20,14403,1973-05-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-42.500,0.164384,21952.0,0.164384,30,3


In [384]:
crsp_merge.columns = [i.lower() for i in crsp_merge.columns]
crsp2 = pd.merge(crsp_merge, ccm2, how='inner', on=['permno'])

MemoryError: 

In [378]:
crsp_merge.shape

(2946158, 12)

In [383]:
crsp_merge[crsp_merge.ticker.isnull()]

,permno,date,ticker,comnam,permco,cusip,prc,ret,shrout,retx,shrcd,exchcd
10243,10058,1974-11-29,NaN,A B K C O INDUSTRIES INC,20,00077410,NaN,0.000000,1470.0,0.000000,10,0
10247,10058,1974-12-31,NaN,A B K C O INDUSTRIES INC,20,00077410,NaN,0.000000,1470.0,0.000000,10,0
12547,62770,1981-04-30,NaN,AMSOUTH BANCORPORATION,25,03216510,-23.875,-0.005208,7393.0,-0.005208,11,3
22722,10621,1983-04-29,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0
22728,10621,1983-05-31,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0
22734,10621,1983-06-30,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0
22740,10621,1983-07-29,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0
22746,10621,1983-08-31,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0
22752,10621,1983-09-30,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0
22758,10621,1983-10-31,NaN,ACCU TEST SYS INC,34,69418810,NaN,0.000000,4249.0,0.000000,10,0


In [380]:
crsp2.head()

,permno,date,ticker,comnam,permco,cusip,prc,ret,shrout,retx,shrcd,exchcd,gvkey,fs_month
0,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,30,3,15510.0,A-Feb
1,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,30,3,15510.0,A-Feb
2,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,30,3,15510.0,A-Feb
3,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,30,3,15510.0,A-Feb
4,14403,1973-01-31,AAGIY,ANGLO AMERICAN GOLD INVT LTD,2,03487050,-27.125,0.179348,21952.0,0.179348,30,3,15510.0,A-Feb
